<a href="https://colab.research.google.com/github/datastax/ragstack-ai/blob/main/examples/notebooks/QA_with_cassio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knowledge Base Search on Proprietary Data powered by Astra DB
This notebook guides you through setting up [RAGStack](https://www.datastax.com/products/ragstack) using [Astra Vector Search](https://docs.datastax.com/en/astra-serverless/docs/vector-search/overview.html), [OpenAI](https://openai.com/about), and [CassIO](https://cassio.org/) to implement a generative Q&A over your own documentation.

## Astra Vector Search
Astra vector search enables developers to search a database by context or meaning rather than keywords or literal values. This is done by using “embeddings”. Embeddings are a type of representation used in machine learning where high-dimensional or complex data is mapped onto vectors in a lower-dimensional space. These vectors capture the semantic properties of the input data, meaning that similar data points have similar embeddings.
Reference: [Astra Vector Search](https://docs.datastax.com/en/astra-serverless/docs/vector-search/overview.html)

## CassIO
CassIO is the ultimate solution for seamlessly integrating Apache Cassandra® with generative artificial intelligence and other machine learning workloads. This powerful Python library simplifies the complicated process of accessing the advanced features of the Cassandra database, including vector search capabilities. With CassIO, developers can fully concentrate on designing and perfecting their AI systems without any concerns regarding the complexities of integration with Cassandra.
Reference: [CassIO](https://cassio.org/)

## OpenAI
OpenAI provides various tools and resources to implement your own Document QA Search system. This includes pre-trained language models like GPT-4, which can understand and generate human-like text. Additionally, OpenAI offers guidelines and APIs to leverage their models for document search and question-answering tasks, enabling developers to build powerful and intelligent Document QA Search applications.
Reference: [OpenAI](https://openai.com/about)

## Demo Summary
ChatGPT excels at answering questions, but only on topics it knows from its training data. It offers you a nice dialog interface to ask questions and get answers.

But what do you do when you have your own documents? How can you leverage the GenAI and LLM models to get insights into those? We can use Retrieval Augmented Generation (RAG) -- think of a Q/A Bot that can answer specific questions over your documentation.

We can do this in two easy steps:
1. Analyzing and storing existing documentation.
2. Providing search capabilities for the model to retrieve your documentation.

This is solved by using LLM models. Ideally you embed the data as vectors and store them in a vector database and then use the LLM models on top of that database.

This notebook demonstrates a basic two-step RAG technique for enabling GPT to answer questions using a library of reference on your own documentation using Astra DB Vector Search.

# Getting Started with this notebook

* Follow [these steps](https://docs.datastax.com/en/astra-serverless/docs/vector-search/overview.html) to create a new vector search enabled database in Astra.
* Generate a new ["Database Administrator" token](https://docs.datastax.com/en/astra-serverless/docs/manage/org/manage-tokens.html).
* Download the secure connect bundle for the database you just created (you can do this from the "Connect" tab of your database).
* You will also need the necessary secret for the LLM provider of your choice:
  * If Open AI, then you will need an [Open AI API Key](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key). This will require an Open AI account with billing enabled.
  * If Vertex AI, you will need a config file.
  * For more details, see [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org.


When you run this notebook, it will ask you to provide each of these items at various steps.

# Setup

Install the following libraries.

In [ ]:
# install required dependencies
! pip install \
    "ragstack-ai" \
    "pypdf"

# Imports

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

In [ ]:
import os
from getpass import getpass

try:
    from google.colab import files
    IS_COLAB = True
except ModuleNotFoundError:
    IS_COLAB = False

# Astra DB configuration, Secure Connection Bundle, and Token

You will need a secure connect bundle and a user with access permission. For demo purposes the "administrator" role will work fine.
More information about how to get the bundle can be found [here](https://docs.datastax.com/en/astra-serverless/docs/connect/secure-connect-bundle.html).

In [ ]:
# Your database's Secure Connect Bundle zip file is needed:
if IS_COLAB:
    print('Please upload your Secure Connect Bundle zipfile: ')
    uploaded = files.upload()
    if uploaded:
        astraBundleFileTitle = list(uploaded.keys())[0]
        ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
    else:
        raise ValueError(
            'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
        )
else:
    # you are running a local-jupyter notebook:
    ASTRA_DB_SECURE_BUNDLE_PATH = input("Please provide the full path to your Secure Connect Bundle zipfile: ")

ASTRA_DB_APPLICATION_TOKEN = getpass("Please provide your Database Token ('AstraCS:...' string): ")
ASTRA_DB_KEYSPACE = input("Please provide the Keyspace name for your Database: ")
# define the table name to be used to store our embeddings, CassIO will create the objects in Astra DB for you.
# To avoid incompatibilities with existing tables, use a new table name. 
ASTRA_DB_TABLE_NAME = input("Please provide the name of the table to be created: ")


# Provide Sample Data
A sample document is provided from CassIO. You may provide your own files instead in the following cell.

In [ ]:
# retrieve the text of a short story that will be indexed in the vector store
! curl https://raw.githubusercontent.com/CassioML/cassio-website/main/docs/frameworks/langchain/texts/amontillado.txt --output amontillado.txt
SAMPLEDATA = ["amontillado.txt"]

In [ ]:
# Alternatively, provide your own file. However, you will want to update your queries to match the content of your file. 

# Upload sample file (Note: this assumes you are on Google Colab. Local Jupyter notebooks can provide the path to their files directly by uncommenting and running just the next line).
# SAMPLEDATA = ["<path_to_file>"]

print('Please upload your own sample file:')
uploaded = files.upload()
if uploaded:
    SAMPLEDATA = uploaded
else:
    raise ValueError(
        'Cannot proceed without Sample Data. Please re-run the cell.'
    )

print(f'Please make sure to change your queries to match the contents of your file!')

# Connect to Astra DB

In [ ]:
# Don't mind the "Closing connection" error after "downgrading protocol..." messages,
# it is really just a warning: the connection will work smoothly.
cluster = Cluster(
    cloud={
        "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
    },
    auth_provider=PlainTextAuthProvider(
        "token",
        ASTRA_DB_APPLICATION_TOKEN,
    ),
)

session = cluster.connect()
keyspace = ASTRA_DB_KEYSPACE

# Read Files, Create Embeddings, Store in Vector DB

CassIO seamlessly integrates with RAGStack and LangChain, offering Cassandra-specific tools for many tasks. In our example we will use vector stores, indexers, embeddings, and queries.

We will use OpenAI for our LLM services. (See [cassio.org](https://cassio.org/start_here/#llm-access) for more details).

In [ ]:
# We will use OpenAI embeddings, so please provide your OpenAI API Key
OPENAI_API_KEY = getpass("Please enter your OpenAI API Key: ")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
# Import the needed libraries and declare the LLM model
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Cassandra
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

# Loop through each file and load it into our vector store
documents = []
for filename in SAMPLEDATA:
  path = os.path.join(os.getcwd(), filename)

  # Supported file types are pdf and txt
  if filename.endswith(".pdf"):
    loader = PyPDFLoader(path)
    new_docs = loader.load_and_split()
    print(f"Processed pdf file: {filename}")
  elif filename.endswith(".txt"):
    loader = TextLoader(path)
    new_docs = loader.load_and_split()
    print(f"Processed txt file: {filename}")
  else:
    print(f"Unsupported file type: {filename}")

  if len(new_docs) > 0:
    documents.extend(new_docs)

cassVStore = Cassandra.from_documents(
  documents=documents,
  embedding=OpenAIEmbeddings(),
  session=session,
  keyspace=ASTRA_DB_KEYSPACE,
  table_name=ASTRA_DB_TABLE_NAME,
)

# empty the list of file names -- we don't want to accidentally load the same files again
SAMPLEDATA = []

print(f"\nProcessing done.")

# Now Query the Data and execute some "searches" against it
First we will start with a similarity search using the Vectorstore's implementation

In [ ]:
# construct your query
prompt = "Who is Luchesi?"

# find matching documentation using similarity search
matched_docs = cassVStore.similarity_search(query=prompt, k=1)

# print out the relevant context that an LLM will use to produce an answer
for i, d in enumerate(matched_docs):
    print(f"\n## Document {i}\n")
    print(d.page_content)

# Finally do a Q/A Search
To be able implement Q/A over documents we need to perform the following steps:

1. Create an Index on top of our vector store
2. Create a Retriever from that Index
3. Ask questions (prompts)!

A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever.

In [ ]:
# Q/A LLM Search
from langchain.chat_models import ChatOpenAI
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

index = VectorStoreIndexWrapper(vectorstore=cassVStore)

# Query the index for relevant vectors to our prompt
prompt = "Who is Luchesi?"
index.query(question=prompt)

In [ ]:
# Alternatively, you can use a retrieval chain with a custom prompt
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate

prompt= """
You are Marv, a sarcastic but factual chatbot.
Context: {context}
Question: {question}
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt)

qa = RetrievalQA.from_chain_type(llm=OpenAI(), retriever=cassVStore.as_retriever(), chain_type_kwargs={"prompt": prompt})

result = qa.run("{question: Who is Luchesi?")
result
